## Libraries

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_percentage_error
from pathlib import Path

In [3]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [4]:
%run ..//..//..//Library//MapeExtended_Library.ipynb 

## Load data

In [5]:
boston = load_boston()

# Initializing the dataframe
data = pd.DataFrame(boston.data)

#Adding the feature names to the dataframe
data.columns = boston.feature_names
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [6]:
#Adding target variable to dataframe
data['PRICE'] = boston.target 

In [7]:
# Spliting target variable and independent variables
X = data.drop(['PRICE'], axis = 1)
y = data['PRICE']

In [8]:
# Matrix with feature transformation
X = pd.read_csv("newBoston.csv").drop(["Unnamed: 0"], axis = 1)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,...,3.323785,7.465139,76.219454,76.219454,-4.485213,-3.176330,97.569478,-3.385,-0.890139,-4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,...,-9.140000,15.561000,57.246040,57.246040,-8.656802,-7.378575,-9.140000,-11.859,-9.140000,-9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,...,8.732227,2.758070,62.356040,62.356040,-3.489308,-2.058993,113.887007,-0.875,4.426930,-4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,...,11.157106,-0.796925,57.959591,57.959591,-2.413381,-1.020291,119.739047,1.118,7.794925,-2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,...,4.303880,7.619455,55.569591,55.569591,-4.792168,-3.369417,76.760299,-3.513,-0.472455,-5.33


In [9]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4)

In [10]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Optimization function

In [11]:
def opt_bas(C, epsilon, lamda, gamma):
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'epsilon' : epsilon, 
        'lamda' : lamda,
        'gamma' : gamma,
    }
    
    # fit and predict
    model = SVR_mapext(**hyperparameters).fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    # get score
    mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
    
    return -mape

## Run optimization

In [12]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [19]:
# Bounded region of parameter space
pbounds = {'C': (38, 67), 'epsilon': (22, 34), 'lamda': (0.0007, 0.011), 'gamma': (0.01, 0.03)}

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=1, verbose=2, 
                                )

# keep data
log_path = Path().resolve() / "Logs" / "Boston_mape_let.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.maximize(init_points=6, n_iter=800)